# Submit Jobrun

## 1. Input Data

In [1]:
import requests
import json
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

job_data = pd.read_csv(r"C:\_Working\PAT_20201019\debug\test_jobrun_input2.csv") #data/test_jobrun.csv')
job_data

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup
0,146975_4418990,146975,500000000,1.0,59462.42,513499.00,4418990,5467663.0,14.0
1,150448_4349144,150448,200000000,1.0,23782.53,369032.00,4349144,21565393.0,14.0
2,157768_4404398,157768,295895093,1.0,50273.97,253952.00,4404398,2856727.0,14.0
3,143094_4443112,143094,128762896,1.0,7890.00,96908.00,4443112,7890.0,14.0
4,140586_4423882,140586,13500000,1.0,14041.10,47985.00,4423882,3100000.0,39.0
...,...,...,...,...,...,...,...,...,...
188356,161645_4342004,161645,340554825,1.0,100000.00,1959921.78,4342004,955000.0,11.0
188357,143271_4387082,143271,117507191,1.0,17491.37,162241.00,4387082,34650321.0,11.0
188358,143219_4319949,143219,250000000,1.0,51804.44,231663.00,4319949,1515284.0,11.0
188359,156824_4404656,156824,100000000,1.0,502997.93,806999.00,4404656,24424723.0,39.0


## 2. Input Parameters

In [2]:
job_para = json.load(open('data/test_jobrun.json'))
print(json.dumps(job_para,indent=2))

{
  "data_source_type": "User_Upload",
  "valid_rules": 1073741826,
  "type_of_rating": "PSOLD",
  "curve_id": 2016,
  "psold": {
    "curve_persp": "Gross",
    "peril_subline": "All_Perils",
    "trend_factor": 1.035,
    "blending": [
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "hpr_blending": false
  },
  "loss_alae_ratio": 1,
  "additional_coverage": 2,
  "deductible_treatment": "Retains_Limit",
  "coverage_type": "Building_Contents_BI",
  "average_accident_date": "1/1/2022",
  "user_name": "Chengyou Xiao",
  "user_email": "chengyou.xiao@guycarp.com"
}


## 3. Request to the service

In [3]:
files = {
    'para': json.dumps(job_para),
    'data': job_data.to_csv(index=False)
}
resp = requests.post("http://localhost:5000/api/jobrun", files=files)
if resp.status_code == 200:
    with ZipFile(BytesIO(resp.content),'r') as zf:
        df_res =  pd.read_csv(zf.open('pat_res.csv'))

df_res.sort_values('PseudoPolicyID')

# 20K records: 6s
# 188K: 28s
# 1m35s vs 26.3s

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup,LossRatio,Policy,EffLmt,AOIr,PolLAS,DedLAS,Premium,sumLAS
172048,138629_4454841,138629,8865000,1.0,1000.0,12538.00,4454841,4882500.0,27.0,1.0,8.866000e+06,14647500.0,34,89512.623964,933.970482,4297.399172,2.584352e+05
172051,138629_4454843,138629,8865000,1.0,1000.0,12538.00,4454843,1382500.0,27.0,1.0,8.866000e+06,4147500.0,29,58505.797325,927.253558,2793.426820,2.584352e+05
172049,138629_4454844,138629,8865000,1.0,2000.0,12538.00,4454844,400000.0,27.0,1.0,8.867000e+06,1200000.0,24,23918.035506,1664.734279,1079.620366,2.584352e+05
172052,138629_4454845,138629,8865000,1.0,1000.0,12538.00,4454845,700000.0,27.0,1.0,8.866000e+06,2100000.0,26,33015.763249,911.961613,1557.518038,2.584352e+05
172050,138629_4454846,138629,8865000,1.0,1000.0,12538.00,4454846,1500000.0,27.0,1.0,8.866000e+06,4500000.0,29,58848.140154,927.253558,2810.035605,2.584352e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69222,240234_4907479,240234,3841845305,1.0,10000.0,512558.79,4907479,23194517.0,14.0,1.0,3.841855e+09,69583551.0,41,47558.521984,5505.543316,2884.834997,7.471701e+06
69255,240234_4907480,240234,3841845305,1.0,10000.0,512558.79,4907480,23194517.0,14.0,1.0,3.841855e+09,69583551.0,41,47558.521984,5505.543316,2884.834997,7.471701e+06
69209,240234_4907481,240234,3841845305,1.0,10000.0,512558.79,4907481,23194517.0,14.0,1.0,3.841855e+09,69583551.0,41,47558.521984,5505.543316,2884.834997,7.471701e+06
69116,240234_4907482,240234,3841845305,1.0,10000.0,512558.79,4907482,23194517.0,14.0,1.0,3.841855e+09,69583551.0,41,47558.521984,5505.543316,2884.834997,7.471701e+06


In [4]:
resp.status_code, resp.headers, df_res.shape


(200,
 {'date': 'Fri, 21 Jan 2022 21:16:38 GMT', 'server': 'uvicorn', 'content-disposition': 'attachment;filename=pat_res.zip', 'access-control-expose-headers': 'Content-Disposition', 'content-type': 'application/zip', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'Transfer-Encoding': 'chunked'},
 (188361, 17))